# Opject Detenction

In [1]:
import os
import pathlib

import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import pandas as pd
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from six.moves.urllib.request import urlopen

import tensorflow as tf
import tensorflow_hub as hub

tf.get_logger().setLevel('ERROR')

/var/folders/xt/dsl_68w51jzftxq18_hpfkhm0000gn/T/ipykernel_61180/3420646353.py:8: DeprecationWarning: scipy.misc is deprecated and will be removed in 2.0.0
  import scipy.misc


## Utilities

In [2]:
models = pd.read_parquet('model_names.parquet', engine="pyarrow")
models = dict(zip(models.ModelName, models.URL))

In [3]:
IMAGES_FOR_TEST = {
  'Beach' : 'models/research/object_detection/test_images/image2.jpg',
  'Dogs' : 'models/research/object_detection/test_images/image1.jpg',
  # By Heiko Gorski, Source: https://commons.wikimedia.org/wiki/File:Naxos_Taverna.jpg
  'Naxos Taverna' : 'https://upload.wikimedia.org/wikipedia/commons/6/60/Naxos_Taverna.jpg',
  # Source: https://commons.wikimedia.org/wiki/File:The_Coleoptera_of_the_British_islands_(Plate_125)_(8592917784).jpg
  'Beatles' : 'https://upload.wikimedia.org/wikipedia/commons/1/1b/The_Coleoptera_of_the_British_islands_%28Plate_125%29_%288592917784%29.jpg',
  # By Américo Toledano, Source: https://commons.wikimedia.org/wiki/File:Biblioteca_Maim%C3%B3nides,_Campus_Universitario_de_Rabanales_007.jpg
  'Phones' : 'https://upload.wikimedia.org/wikipedia/commons/thumb/0/0d/Biblioteca_Maim%C3%B3nides%2C_Campus_Universitario_de_Rabanales_007.jpg/1024px-Biblioteca_Maim%C3%B3nides%2C_Campus_Universitario_de_Rabanales_007.jpg',
  # Source: https://commons.wikimedia.org/wiki/File:The_smaller_British_birds_(8053836633).jpg
  'Birds' : 'https://upload.wikimedia.org/wikipedia/commons/0/09/The_smaller_British_birds_%288053836633%29.jpg',
}

In [4]:
def load_images_into_numpy_array(path):
    image = None
    if(path.startswith('http')):
        response = urlopen(path)
        image_data = response.read()
        image_data = BytesIO(image_data)
        image = Image.open(image_data)
    else:
        image_data = tf.io.gfile.GFile(path, 'rb').read()
        image = Image.open(BytesIO(image_data))

    (im_width, im_height) = image.size
    return (np.array(image.getdata())
            .reshape(1, im_height, im_width, 3)).astype(np.uint8)
    
    COCO17_HUMAN_POSE_KEYPOINTS = [
                                 (0, 1),
                                 (0, 2),
                                 (1, 3),
                                 (2, 4),
                                 (0, 5),
                                 (0, 6),
                                 (5, 7),
                                 (7, 9),
                                 (6, 8),
                                 (8, 10),
                                 (5, 6),
                                 (5, 11),
                                 (6, 12),
                                 (11, 12),
                                 (11, 13),
                                 (13, 15),
                                 (12, 14),
                                 (14, 16)]


In [5]:
! git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 4352, done.
remote: Counting objects: 100% (4352/4352), done.06/4352)
remote: Compressing objects: 100% (3161/3161), done.
error: RPC failed; curl 92 HTTP/2 stream 5 was not closed cleanly: CANCEL (err 8)12% (543/4352), 1.04 MiB | 14.00 KiB/sReceiving objects:  13% (566/4352), 1.07 MiB | 15.00 KiB/sReceiving objects:  13% (574/4352), 1.09 MiB | 17.00 KiB/sReceiving objects:  13% (591/4352), 1.14 MiB | 20.00 KiB/sReceiving objects:  14% (610/4352), 1.18 MiB | 21.00 KiB/sReceiving objects:  14% (636/4352), 1.22 MiB | 23.00 KiB/sReceiving objects:  15% (666/4352), 1.28 MiB | 22.00 KiB/sReceiving objects:  16% (703/4352), 1.31 MiB | 22.00 KiB/sReceiving objects:  17% (740/4352), 1.34 MiB | 22.00 KiB/sReceiving objects:  17% (752/4352), 1.36 MiB | 20.00 KiB/sReceiving objects:  17% (758/4352), 1.37 MiB | 15.00 KiB/sReceiving objects:  18% (784/4352), 1.39 MiB | 14.00 KiB/sReceiving objects:  18% (804/4352), 1.43 MiB | 13.00 KiB/sReceivin

In [ ]:
! sudo apt install -y protobuf-compiler
! cd models/research/
! protoc object_detection/protos/*.proto --python_out=.
! cp object_detection/packages/tf2/setup.py .
! python3 -m pip install .

Password: